## Camera Calibration

## EDITED FROM KAGGLE by safal

In [ ]:
import git
import shutil
def clone():
    # URL of the repository to clone
    
    repo_url = 'https://github.com/markakash/BigTree.git'

    # The path where the repository will be cloned
    clone_path = '/kaggle/working/BigTree'

    # Name of the branch you want to clone
    branch_name = 'kaggle'

    # Cloning the specific branch
    git.Repo.clone_from(repo_url, clone_path, branch=branch_name)

### Variables

In [ ]:
VIDEO_CALIBRATION_PATH = "kaggle/input/video/calibration.MP4"
EXTRACT_FRAMES = True
CHECKERBOARD_SIZE = (6, 8)

### Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from termcolor import colored
import imageio
import json

In [ ]:
if EXTRACT_FRAMES:
    reader = imageio.get_reader(VIDEO_CALIBRATION_PATH)

    try:
        for i, frame in enumerate(reader):
            cv2.imwrite(f'kaggle/working/calibration_frames/frame_{i}.png', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            print(f'Read frame: {i}', end='\r')
    except Exception as e:
        print(f'An error occurred: {e}')
    finally:
        reader.close()

### Calibration matrix

In [ ]:
# Init our checkerboard in 3 dimensions
# checkerboard_world_space = np.zeros((CHECKERBOARD_SIZE[0]*CHECKERBOARD_SIZE[1],3), np.float32)
# checkerboard_world_space[:,:2] = np.mgrid[0:CHECKERBOARD_SIZE[0],0:CHECKERBOARD_SIZE[1]].T.reshape(-1,2)

# objpoints = [] # 3D points in real world space
# imgpoints = [] # 2D points in image plane

# for file in glob.glob('assets/calibration_frames/*.png'):
#     img = cv2.imread(file)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     # Find the chess board corners, thank you cv2 for providing such a useful function
#     ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD_SIZE, None)

#     if ret == True:
#         objpoints.append(checkerboard_world_space)
#         imgpoints.append(corners)

# # Calibrate camera
# ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# print(colored("Camera matrix:\n\n", "green"), mtx, "\n")
# print(colored("Distortion coefficients:\n\n", "green"), dist)

In [ ]:
def calibrate_camera_with_subsets(subset_size=100, n_experiments=20):
    files = glob.glob('kaggle/working/calibration_frames/*.png')
    np.random.seed(42)
        
    camera_matrices = []
    distortion_coefficients = []
    
    best_mtx = None
    best_dist = None
    lowest_error = np.inf
    
    for _ in range(n_experiments):
        random_selected_files = np.random.choice(files, size=subset_size, replace=False)
        
        objpoints = [] # 3D points in real world space
        imgpoints = [] # 2D points in image plane
        
        checkerboard_world_space = np.zeros((CHECKERBOARD_SIZE[0]*CHECKERBOARD_SIZE[1], 3), np.float32)
        checkerboard_world_space[:, :2] = np.mgrid[0:CHECKERBOARD_SIZE[0], 0:CHECKERBOARD_SIZE[1]].T.reshape(-1, 2)
        
        for file in random_selected_files:
            img = cv2.imread(file)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Find the chess board corners
            ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD_SIZE, None)
            
            # If found, add object points, image points
            if ret == True:
                objpoints.append(checkerboard_world_space)
                imgpoints.append(corners)
        
        # Calibrate camera
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
        # Store calibration parameters
        if ret:
            camera_matrices.append(mtx)
            distortion_coefficients.append(dist)
            
            # Calculate re-projection error
            mean_error = 0
            for i in range(len(objpoints)):
                imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
                mean_error += error
            
            if(mean_error < lowest_error):
                lowest_error = mean_error
                best_mtx = mtx
                best_dist = dist
    
    # Numpy is more efficient...
    camera_matrices = np.array(camera_matrices)
    distortion_coefficients = np.array(distortion_coefficients)
    
    # Compute standard deviation
    camera_matrices_std = np.std(camera_matrices, axis=0)
    distortion_coefficients_std = np.std(distortion_coefficients, axis=0)
    
    return best_mtx, best_dist, camera_matrices_std, distortion_coefficients_std

In [ ]:
best_mtx, best_dist, camera_matrices_std, distortion_coefficients_std = calibrate_camera_with_subsets()
print("\nStandard deviation of camera matrices:\n", camera_matrices_std)
print("\nStandard deviation of distortion coefficients:\n", distortion_coefficients_std)
print("\nBest camera matrix:\n", best_mtx)
print("\nBest distortion coefficients:\n", best_dist)

In [ ]:
# 
def match_frames(img1, img2):
    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Create BFMatcher object with default params
    bf = cv2.BFMatcher()

    # Match descriptors
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test to find good matches
    good_matches = []
    pts1 = []
    pts2 = []

    for m, n in matches:
        if m.distance < 0.75 * n.distance:  # only accept matchs that are significantly better than the second best match
            good_matches.append(m)
            pts1.append(keypoints1[m.queryIdx].pt)
            pts2.append(keypoints2[m.trainIdx].pt)

    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)

    # Compute the fundamental matrix
    fundamental_matrix, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

    return pts1, pts2, fundamental_matrix

def compute_essential_matrix(fundamental_matrix, calibration_matrix):
    E = calibration_matrix.T @ fundamental_matrix @ calibration_matrix
    return E

In [ ]:
# This is an implementation of the 5-point algorithm for computing the essential matrix from a set of 5 correspondences.
# This isn't being used. We are using the match_frame one.
def match_sift_descriptors(img1, img2, drawepilines=False, threshold=0.75):
    """Matches SIFT descriptors between two images and optionally draws epilines."""
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    good_matches = []
    pts1 = []
    pts2 = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    if drawepilines:
        F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_LMEDS)
        pts1 = pts1[mask.ravel() == 1]
        pts2 = pts2[mask.ravel() == 1]
        lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
        lines1 = lines1.reshape(-1, 3)
        img1_with_lines, img2_with_lines = draw_epilines(img1, img2, lines1, pts1, pts2)
        return pts1, pts2, F, img1_with_lines, img2_with_lines
    else:
        img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        return img_matches

In [38]:
# Setup video capture
cap = cv2.VideoCapture('kaggle/input/video/eastbound_20240319_short_20fps_compressed.MP4')
calibration_matrix = best_mtx

results = {}
frame_index = 0
ret, previous_frame = cap.read()
previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, current_frame = cap.read()
    if not ret:
        break
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # Compute the fundamental matrix
    pts1, pts2, fundamental_matrix = match_frames(previous_frame_gray, current_frame_gray)

    # Compute the essential matrix
    essential_matrix = compute_essential_matrix(fundamental_matrix, calibration_matrix)

    # Decompose the essential matrix
    R1, R2, t = cv2.decomposeEssentialMat(essential_matrix)

    # Store results in the dictionary
    results[f"Frame {frame_index}-{frame_index+1}"] = {
        "Fundamental Matrix": fundamental_matrix,
        "Essential Matrix": essential_matrix,
        "Rotation Matrix 1": R1,
        "Rotation Matrix 2": R2,
        "Translation Vector": t.ravel().tolist()
    }

    # Update the previous frame
    previous_frame_gray = current_frame_gray
    frame_index += 1

cap.release()

# Save results to a JSON file
with open('output_matrices.json', 'w') as file:
    json.dump(results, file, indent=4)

KeyboardInterrupt: 

## Results
```py
Standard deviation of camera matrices:
 [[3.70849068 0.         5.82549989]
 [0.         2.98369589 9.05924656]
 [0.         0.         0.        ]]

Standard deviation of distortion coefficients:
 [[0.00296308 0.00773438 0.00029421 0.00040361 0.00619906]]

Best camera matrix:
 [[1.41527706e+03 0.00000000e+00 1.32534026e+03]
 [0.00000000e+00 1.42789878e+03 8.00498436e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Best distortion coefficients:
 [[-0.28111115  0.14996815  0.00110429  0.00084166 -0.05461088]]
 ```

In [ ]:
frame_test = cv2.imread('/kaggle/working/calibration_frames/frame_555.png')

# Undistort image
undistorted_img = cv2.undistort(frame_test, best_mtx, best_dist, None, best_mtx)

fig, axs = plt.subplots(1, 2, figsize=(30, 20))
axs[0].imshow(frame_test)
axs[0].set_title('Original Image')
axs[0].axis('off')

axs[1].imshow(undistorted_img)
axs[1].set_title('Undistorted Image')
axs[1].axis('off')

plt.show()